In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras
# from tensorflow.keras.preprocessing.text import Tokenizer
# from underthesea import word_tokenize

In [ ]:
col_list = ["stt", "word", "lucky_number"]
dreamwords = pd.read_csv("dreambook_preprocessed.csv", usecols=col_list)
dreamwords_list = list(dreamwords["word"])
number_list = list(dreamwords["lucky_number"])

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = len(number_list)  # Number of samples to train on.

# Path to the data txt file on disk.
# data_path = "fra.txt"

In [ ]:
# Vectorize the data.
unique_tokens = set()
unique_labels = set()
input_texts = list()
target_texts = list()

# for sentence in dreamwords_list:
#     if type(sentence) is float:
#         sentence = ""
#     input_texts.append(word_tokenize(sentence, format="text"))

for numbers in number_list:
    target_texts.append(numbers.replace("-", " "))

input_texts = dreamwords_list

# [[unique_tokens.add(token) for token in tokens.split(" ")] for tokens in input_texts]
# max_sentence_len = max([len(tokens.split(" ")) for tokens in input_texts])

[[unique_tokens.add(token) for token in tokens] for tokens in input_texts]
max_sentence_len = max([len(tokens) for tokens in input_texts]) 

[[unique_labels.add(number) for number in numbers] for numbers in target_texts]
max_label_len = max([len(numbers) for numbers in target_texts])

num_encoder_tokens = len(unique_tokens)
num_decoder_tokens = len(unique_labels)
max_encoder_seq_length = max_sentence_len
max_decoder_seq_length = max_label_len

input_characters = list(unique_tokens)
target_characters = list(unique_labels)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

print("Number of samples:", len(number_list))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)

decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [55]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    # target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    # stop_condition = False
    # decoded_sentence = ""
    
    # while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # Exit condition: either hit max length
    # or find stop character.
    # if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
    #     stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.0

    # Update states
    states_value = [h, c]
    
    return decoded_sentence

def encode_input_text(sentence):
    encoder_input_data = np.zeros(
        (len(sentence), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
    )
    for t, char in enumerate(sentence):
        encoder_input_data[0, t, input_token_index[char]] = 1.0
    
    return encoder_input_data

In [56]:
sentence = "chuột đồng chạy quanh sân"

input_seq = encode_input_text(sentence)
decoded_sentence = decode_sequence(input_seq)

print("Input sentence:", sentence)
print("Decoded sentence:", decoded_sentence)

[array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])]
1
2


ValueError: Data cardinality is ambiguous:
  x sizes: 1, 25, 25
Make sure all arrays contain the same number of samples.